<a href="https://colab.research.google.com/github/jromerooo2/feature_extraction/blob/main/Audio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! git clone https://github.com/v-iashin/video_features.git
! pip install omegaconf==2.0.6
! pip install resampy

Cloning into 'video_features'...
remote: Enumerating objects: 1299, done.
remote: Counting objects: 100% (420/420), done.
remote: Compressing objects: 100% (189/189), done.
remote: Total 1299 (delta 264), reused 322 (delta 215), pack-reused 879
Receiving objects: 100% (1299/1299), 288.63 MiB | 18.27 MiB/s, done.
Resolving deltas: 100% (671/671), done.
Updating files: 100% (177/177), done.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 67.0 MB/s eta 0:00:00


In [ ]:
%cd video_features

/content/video_features


To Properly Run: Please drop the files under the folder named sample inside of the video_features git clone.

In [ ]:
import os

folder_path = 'sample/'

files = os.listdir(folder_path)

output_file = 'file_paths.txt'

with open(output_file, 'w') as f:
    for file_name in files:
        file_path = os.path.join(folder_path, file_name)
        f.write(file_path + '\n')

print("everything written to:", output_file)


everything written to: file_paths.txt


In [ ]:
from models.vggish.extract_vggish import ExtractVGGish
from utils.utils import build_cfg_path
import numpy as np
from omegaconf import OmegaConf
import torch
import os

device = 'cuda' if torch.cuda.is_available() else 'cpu'
torch.cuda.get_device_name(0)

'Tesla T4'

In [ ]:
feature_type = 'vggish'

args = OmegaConf.load(build_cfg_path(feature_type))

extractor = ExtractVGGish(args)

for i,dir in enumerate(os.listdir('./sample')):
  if dir.endswith('.wav'):
    feature_dict = extractor.extract('./sample/' + dir)
    #change the '' with the desired path to save at
    np.save('./deep_{}_features.npy'.format(i), np.array(feature_dict, dtype=object), allow_pickle=True)

# HandCrafted Extractions: Using OpenSmile

In [ ]:
%pip install opensmile

%pip install audb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 35.5 MB/s eta 0:00:00


In [ ]:
import os
import time
import numpy as np
import pandas as pd
import audb
import audiofile
import opensmile


smile = opensmile.Smile(
    feature_set=opensmile.FeatureSet.ComParE_2016,
    feature_level=opensmile.FeatureLevel.Functionals,
)
#TODO: Change this to audformat to have a better organization of the files

audio_folder = 'sample'
all_audio = [f for f in os.listdir(audio_folder) if f.endswith('.wav')]
all_audio.sort()

for file in all_audio:
    dur = audiofile.duration(os.path.join(audio_folder,file))
    print(dur)
    all_features= pd.DataFrame([])

    list_steps = np.arange(0.0, dur)
    for i in range(len(list_steps)-1):
        feature = smile.process_file(
                os.path.join(audio_folder,file),
                start=list_steps[i],
                end=list_steps[i+1],
        )
        all_features = feature if all_features.empty else all_features.append(feature)
    #saves 3 types of files: npy, pickle and csv
    #replace with desired folder
    save_file_name =  os.path.join('','./','{}_handcrafted.'.format(file))
    np.save(save_file_name , np.array(all_features,dtype=object), allow_pickle=True)
    all_features.to_pickle(save_file_name+'pkl')
    all_features.to_csv(save_file_name+'csv')

1.7724036281179139
29.983061224489795


<ipython-input-22-8a28a9b42c02>:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_features = feature if all_features.empty else all_features.append(feature)
<ipython-input-22-8a28a9b42c02>:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_features = feature if all_features.empty else all_features.append(feature)
<ipython-input-22-8a28a9b42c02>:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_features = feature if all_features.empty else all_features.append(feature)
<ipython-input-22-8a28a9b42c02>:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_features = feature if all_features.empty else all_features.append(feature)
